# Run CRISPR-Correct on CD19 data

This will use CRISPR-Correct version 0.0.41, which may have an different interface than the latest version.

### Download CD19 Example Data

As an example, download the ABE8e and EvoDDA data from GEO that was used in the CRISPR-CLEAR manuscript: [GSE278581](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE278581)
- GSM8549767	ABE8e, replicate 1, presort, pooled, gRNA
- GSM8549768	ABE8e, replicate 1, CD19 positive, pooled, gRNA
- GSM8549769	ABE8e, replicate 1, CD19 negative, pooled, gRNA
- GSM8549770	ABE8e, replicate 2, presort, pooled, gRNA
- GSM8549771	ABE8e, replicate 2, CD19 positive, pooled, gRNA
- GSM8549772	ABE8e, replicate 2, CD19 negative, pooled, gRNA
- GSM8549773	ABE8e, replicate 3, presort, pooled, gRNA
- GSM8549774	ABE8e, replicate 3, CD19 positive, pooled, gRNA
- GSM8549775	ABE8e, replicate 3, CD19 negative, pooled, gRNA
- GSM8549776	ABE8e, plasmid, pooled, gRNA
- GSM8549777	evoCDA, replicate 1, presort, pooled, gRNA
- GSM8549778	evoCDA, replicate 1, CD19 positive, pooled, gRNA
- GSM8549779	evoCDA, replicate 1, CD19 negative, pooled, gRNA
- GSM8549780	evoCDA, replicate 2, presort, pooled, gRNA
- GSM8549781	evoCDA, replicate 2, CD19 positive, pooled, gRNA
- GSM8549782	evoCDA, replicate 2, CD19 negative, pooled, gRNA
- GSM8549783	evoCDA, replicate 3, presort, pooled, gRNA
- GSM8549784	evoCDA, replicate 3, CD19 positive, pooled, gRNA
- GSM8549785	evoCDA, replicate 3, CD19 negative, pooled, gRNA
- GSM8549786	evoCDA, plasmid, pooled, gRNA

Alternatively and perhaps easier, you can download the files from [Zenodo](https://zenodo.org/records/13737880):
- CRISPR-CLEAR-data/data/raw_FASTQs/guide_sequencing/ABE8e_fastqs.zip
- CRISPR-CLEAR-data/data/raw_FASTQs/guide_sequencing/EvoCDA_fastqs.zip

Download the files and unzip.


### Install CRISPR_ambiguous_mapping

In [ ]:
reinstall = True

if reinstall:    
    version = "0.0.41"
    !pip uninstall -y crispr-ambiguous-mapping
    !pip install --no-cache-dir crispr-ambiguous-mapping==$version

### Import Packages

In [3]:
%load_ext autoreload
%autoreload 2

import crispr_ambiguous_mapping

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Set inputs

In [5]:
#  Guide DF Filenames
CD19_guide_library_fn = "CD19_sgRNA_count_libraries_DS.txt"

# Load CD19 Library
CD19_guide_library_raw_df = pd.read_table(CD19_guide_library_fn, encoding='utf-8')
CD19_guide_library_raw_df.columns = [colname.strip() for colname in CD19_guide_library_raw_df.columns]

In [6]:
# Prepare dataframe for CRISPR-SURF input

CD19_guide_library_SURFinput_df = pd.DataFrame({"Chr": CD19_guide_library_raw_df["chromosome_#"],
            "Start": CD19_guide_library_raw_df["start"],
            "Stop": CD19_guide_library_raw_df["end"],
            "sgRNA_Sequence": CD19_guide_library_raw_df["sgRNA"],
            "Strand": CD19_guide_library_raw_df["strand"],
            "sgRNA_Type_ABE": np.where(CD19_guide_library_raw_df["start"].isna(), "negative_control", "observation"),
            "sgRNA_Type_CBE": np.where(CD19_guide_library_raw_df["start"].isna(), "negative_control", "observation")})

CD19_guide_library_SURFinput_df["Chr"] = CD19_guide_library_SURFinput_df["Chr"].str.replace(",","_")
CD19_guide_library_SURFinput_df.to_csv("CD19_guide_library_SURFinput.csv", index=False)

In [7]:
# Set predicted cutsite for each guide

CD19_guide_library_SURFinput_df_cutsite = CD19_guide_library_SURFinput_df.copy()
CD19_guide_library_SURFinput_df_cutsite["Cutsite"] = np.where(CD19_guide_library_SURFinput_df_cutsite["Strand"] == "+", CD19_guide_library_SURFinput_df_cutsite["Start"]+6, CD19_guide_library_SURFinput_df_cutsite["Start"]+14)

### Get the demultiplexed fileames

In [4]:
# REPLACE ABE8e DIRECTORY WITH DOWNLOADED DIRECTORY
cd19_abe8e_redo_directory = "./ABE8e_fastqs"

cd19_abe8e_redo_presort_fns = ["AAGTAGAG_AAGTAGAG_R1-CD19_presort_1_ABE8.fastq.gz",
                              "AAGTAGAG_CATGATCG_R1-CD19_presort_2_ABE8.fastq.gz",
                              "AAGTAGAG_AACGCATT_R1-CD19_presort_3_ABE8.fastq.gz"]
cd19_abe8e_redo_low_fns = ["AAGTAGAG_ACACGATC_R1-CD19_low_1_ABE8.fastq.gz",
                          "AAGTAGAG_CGTTACCA_R1-CD19_low_2_ABE8.fastq.gz",
                          "AAGTAGAG_AGGTAAGG_R1-CD19_low_3_ABE8.fastq.gz"]
cd19_abe8e_redo_plasmid_fns = ["CGCGCGGT_AGGTAAGG_R1_CD19_plasmid_pool.fastq.gz",
                              "CGCGCGGT_AGGTAAGG_R1_CD19_plasmid_pool.fastq.gz",
                              "CGCGCGGT_AGGTAAGG_R1_CD19_plasmid_pool.fastq.gz"]
cd19_abe8e_redo_high_fns = ["AAGTAGAG_CGCGCGGT_R1-CD19_high_1_ABE8.fastq.gz",
                           "AAGTAGAG_TCCTTGGT_R1-CD19_high_2_ABE8.fastq.gz",
                           "AAGTAGAG_AACAATGG_R1-CD19_high_3_ABE8.fastq.gz"]


# REPLACE EvoCDA DIRECTORY WITH DOWNLOADED DIRECTORY
cd19_evocda_directory = "./EvoCDA_fastqs"

cd19_evocda_presort_fns = ["EvoCDA_Presort_0.R1.fastq.gz",
                              "EvoCDA_Presort_1.R1.fastq.gz",
                              "EvoCDA_Presort_2.R1.fastq.gz"]
cd19_evocda_low_fns = ["EvoCDA_Low_0.R1.fastq.gz",
                          "EvoCDA_Low_1.R1.fastq.gz",
                          "EvoCDA_Low_2.R1.fastq.gz"]
cd19_evocda_plasmid_fns = ["EvoCDA_Plasmid_0.R1.fastq.gz",
                              "EvoCDA_Plasmid_1.R1.fastq.gz",
                              "EvoCDA_Plasmid_2.R1.fastq.gz"]
cd19_evocda_high_fns = ["EvoCDA_High_0.R1.fastq.gz",
                           "EvoCDA_High_1.R1.fastq.gz",
                           "EvoCDA_High_2.R1.fastq.gz"]

In [16]:
cd19_guide_series = CD19_guide_library_SURFinput_df_cutsite["sgRNA_Sequence"]
cd19_guide_series.index = cd19_guide_series.values

### Perform count for the ABE8e (self-editing aware)

In [17]:
# FUNCTION TO PERFORM COUNT
def perform_count(input_fn,output_fn, guide_series, counts_output_dir="./", hamming_distance=11):
    print("Reading input: " + input_fn)
    count_result = crispr_ambiguous_mapping.get_guide_counts_from_fastq(guide_series, input_fn, hamming_threshold_strict=hamming_distance, parse_left_flank=False, parse_flank_sequence="GTTTTA", cores=20)
    crispr_ambiguous_mapping.save_or_load_pickle(f"{counts_output_dir}", output_fn, py_object=count_result, date_string="20230426")

In [22]:
%%time

rerun = True

# PERFORM COUNT FOR ALL SAMPLES
output_dir = "./"
if rerun:
    [perform_count(input_fn=f"{cd19_abe8e_redo_directory}/{fn}",output_fn=f"cd19_abe8e_presort_count_{index}", counts_output_dir=output_dir,guide_series=cd19_guide_series) for index, fn in enumerate(cd19_abe8e_redo_presort_fns)]
    [perform_count(input_fn=f"{cd19_abe8e_redo_directory}/{fn}",output_fn=f"cd19_abe8e_low_count_{index}", counts_output_dir=output_dir, guide_series=cd19_guide_series) for index, fn in enumerate(cd19_abe8e_redo_low_fns)]
    [perform_count(input_fn=f"{cd19_abe8e_redo_directory}/{fn}",output_fn=f"cd19_abe8e_plasmid_count_{index}", counts_output_dir=output_dir, guide_series=cd19_guide_series) for index, fn in enumerate(cd19_abe8e_redo_plasmid_fns)]
    [perform_count(input_fn=f"{cd19_abe8e_redo_directory}/{fn}",output_fn=f"cd19_abe8e_high_count_{index}", counts_output_dir=output_dir, guide_series=cd19_guide_series) for index, fn in enumerate(cd19_abe8e_redo_high_fns)]
    
    [perform_count(input_fn=f"{cd19_evocda_directory}/{fn}",output_fn=f"cd19_evocda_presort_count_{index}", counts_output_dir=output_dir, guide_series=cd19_guide_series) for index, fn in enumerate(cd19_evocda_presort_fns)]
    [perform_count(input_fn=f"{cd19_evocda_directory}/{fn}",output_fn=f"cd19_evocda_low_count_{index}", counts_output_dir=output_dir, guide_series=cd19_guide_series) for index, fn in enumerate(cd19_evocda_low_fns)]
    [perform_count(input_fn=f"{cd19_evocda_directory}/{fn}",output_fn=f"cd19_evocda_plasmid_count_{index}", counts_output_dir=output_dir, guide_series=cd19_guide_series) for index, fn in enumerate(cd19_evocda_plasmid_fns)]
    [perform_count(input_fn=f"{cd19_evocda_directory}/{fn}",output_fn=f"cd19_evocda_high_count_{index}", counts_output_dir=output_dir, guide_series=cd19_guide_series) for index, fn in enumerate(cd19_evocda_high_fns)]

CPU times: user 1 µs, sys: 5 µs, total: 6 µs
Wall time: 11.4 µs


In [6]:
# FUNCTION TO READ IN THE COUNT RESULT
def read_count(output_fn, counts_output_dir="./"):
    count_result = crispr_ambiguous_mapping.save_or_load_pickle(f"{counts_output_dir}", output_fn, date_string="20230426")
    return count_result

In [ ]:
# READ IN THE COUNT RESULT
cd19_abe8e_presort_count_result_list = [read_count(output_fn=f"cd19_abe8e_presort_count_{index}", counts_output_dir=output_dir) for index, fn in enumerate(cd19_abe8e_redo_presort_fns)]
cd19_abe8e_low_count_result_list = [read_count(output_fn=f"cd19_abe8e_low_count_{index}", counts_output_dir=output_dir) for index, fn in enumerate(cd19_abe8e_redo_low_fns)]
cd19_abe8e_plasmid_count_result_list = [read_count(output_fn=f"cd19_abe8e_plasmid_count_{index}", counts_output_dir=output_dir) for index, fn in enumerate(cd19_abe8e_redo_plasmid_fns)]
cd19_abe8e_high_count_result_list = [read_count(output_fn=f"cd19_abe8e_high_count_{index}", counts_output_dir=output_dir) for index, fn in enumerate(cd19_abe8e_redo_high_fns)]

cd19_evocda_presort_count_result_list = [read_count(output_fn=f"cd19_evocda_presort_count_{index}", counts_output_dir=output_dir) for index, fn in enumerate(cd19_evocda_presort_fns)]
cd19_evocda_low_count_result_list = [read_count(output_fn=f"cd19_evocda_low_count_{index}", counts_output_dir=output_dir) for index, fn in enumerate(cd19_evocda_low_fns)]
cd19_evocda_plasmid_count_result_list = [read_count(output_fn=f"cd19_evocda_plasmid_count_{index}", counts_output_dir=output_dir) for index, fn in enumerate(cd19_evocda_plasmid_fns)]
cd19_evocda_high_count_result_list = [read_count(output_fn=f"cd19_evocda_high_count_{index}", counts_output_dir=output_dir) for index, fn in enumerate(cd19_evocda_high_fns)]

In [43]:
# Show the mapping percentage
print(f"ABE8e presort: {[count_result[2]['percent_mapped'] for count_result in cd19_abe8e_presort_count_result_list]}")
print(f"ABE8e low: {[count_result[2]['percent_mapped'] for count_result in cd19_abe8e_low_count_result_list]}")
print(f"ABE8e plasmid: {[count_result[2]['percent_mapped'] for count_result in cd19_abe8e_plasmid_count_result_list]}")
print(f"ABE8e high: {[count_result[2]['percent_mapped'] for count_result in cd19_abe8e_high_count_result_list]}")

print(f"evoCDA presort: {[count_result[2]['percent_mapped'] for count_result in cd19_evocda_presort_count_result_list]}")
print(f"evoCDA low: {[count_result[2]['percent_mapped'] for count_result in cd19_evocda_low_count_result_list]}")
print(f"evoCDA plasmid: {[count_result[2]['percent_mapped'] for count_result in cd19_evocda_plasmid_count_result_list]}")
print(f"evoCDA high: {[count_result[2]['percent_mapped'] for count_result in cd19_evocda_high_count_result_list]}")

ABE8e presort: [0.9260519115465721, 0.9294988553983029, 0.9330139311983668]
ABE8e low: [0.9410854701165292, 0.8198730576009812, 0.937903920670365]
ABE8e plasmid: [0.9634341602988671, 0.9634341602988671, 0.9634341602988671]
ABE8e high: [0.9451304230573598, 0.9358039100338151, 0.9331782180155964]
evoCDA presort: [0.912013822906674, 0.8659725792176545, 0.9072118619683116]
evoCDA low: [0.8633668249045483, 0.7524936663391386, 0.6858312465457714]
evoCDA plasmid: [0.9230416249892666, 0.9230416249892666, 0.9230416249892666]
evoCDA high: [0.8037354255987659, 0.8247871403838533, 0.80906838726479]


In [182]:
# Show the mapped read counts
print(f"ABE8e presort: {[count_result[2]['total_guide_counts'] * count_result[2]['percent_mapped'] for count_result in cd19_abe8e_presort_count_result_list]}")
print(f"ABE8e low: {[count_result[2]['total_guide_counts']  * count_result[2]['percent_mapped'] for count_result in cd19_abe8e_low_count_result_list]}")
print(f"ABE8e plasmid: {[count_result[2]['total_guide_counts']  * count_result[2]['percent_mapped'] for count_result in cd19_abe8e_plasmid_count_result_list]}")
print(f"ABE8e high: {[count_result[2]['total_guide_counts']  * count_result[2]['percent_mapped'] for count_result in cd19_abe8e_high_count_result_list]}")

print(f"evoCDA presort: {[count_result[2]['total_guide_counts']  * count_result[2]['percent_mapped'] for count_result in cd19_evocda_presort_count_result_list]}")
print(f"evoCDA low: {[count_result[2]['total_guide_counts']  * count_result[2]['percent_mapped'] for count_result in cd19_evocda_low_count_result_list]}")
print(f"evoCDA plasmid: {[count_result[2]['total_guide_counts']  * count_result[2]['percent_mapped'] for count_result in cd19_evocda_plasmid_count_result_list]}")
print(f"evoCDA high: {[count_result[2]['total_guide_counts']  * count_result[2]['percent_mapped'] for count_result in cd19_evocda_high_count_result_list]}")

ABE8e presort: [17842282.0, 17295913.0, 13438933.0]
ABE8e low: [23566556.0, 13749891.0, 15117383.0]
ABE8e plasmid: [11639593.0, 11639593.0, 11639593.0]
ABE8e high: [14922703.0, 14895859.0, 18162792.0]
evoCDA presort: [8568752.0, 8823573.0, 7245355.0]
evoCDA low: [7692601.0, 6967533.0, 4772605.0]
evoCDA plasmid: [10437835.0, 10437835.0, 10437835.0]
evoCDA high: [7547496.0, 7973454.0, 5766088.0]


In [183]:
# Show another way to get the total mapped reads by summing over count series
print(f"ABE8e presort: {[sum(count_result[0]) for count_result in cd19_abe8e_presort_count_result_list]}")
print(f"ABE8e low: {[sum(count_result[0]) for count_result in cd19_abe8e_low_count_result_list]}")
print(f"ABE8e plasmid: {[sum(count_result[0]) for count_result in cd19_abe8e_plasmid_count_result_list]}")
print(f"ABE8e high: {[sum(count_result[0]) for count_result in cd19_abe8e_high_count_result_list]}")

print(f"evoCDA presort: {[sum(count_result[0]) for count_result in cd19_evocda_presort_count_result_list]}")
print(f"evoCDA low: {[sum(count_result[0]) for count_result in cd19_evocda_low_count_result_list]}")
print(f"evoCDA plasmid: {[sum(count_result[0]) for count_result in cd19_evocda_plasmid_count_result_list]}")
print(f"evoCDA high: {[sum(count_result[0]) for count_result in cd19_evocda_high_count_result_list]}")

ABE8e presort: [17842282, 17295913, 13438933]
ABE8e low: [23566556, 13749891, 15117383]
ABE8e plasmid: [11639593, 11639593, 11639593]
ABE8e high: [14922703, 14895859, 18162792]
evoCDA presort: [8568752, 8823573, 7245355]
evoCDA low: [7692601, 6967533, 4772605]
evoCDA plasmid: [10437835, 10437835, 10437835]
evoCDA high: [7547496, 7973454, 5766088]


In [186]:
# Show example of a count series for ABE8e presort replicate 1
cd19_abe8e_presort_count_result_list[0][0]

protospacer         
GGGGAATGACATGCTCTAGT     24716
GAATGACATGCTCTAGTGAA     11750
TGACATGCTCTAGTGAAAGC      7392
CATGCTCTAGTGAAAGCCAG     44819
GCTCTAGTGAAAGCCAGTCT    100774
                         ...  
CCACCTTATATTCCCAGGGC     41463
TATAAGGTGGTCCCAGCTCG     31665
CGGGGACACAGGATCCCTGG     38163
TGCTGTCCTGAAGTGGACAT     69199
GTGGACATAGGGGCCCGGGT     17320
Name: sgRNA_Sequence, Length: 206, dtype: int64

#### Prepare the count dataframe

In [50]:
# Load the counts
cd19_abe8e_presort_count_df = pd.concat([count_result[0] for count_result in cd19_abe8e_presort_count_result_list], axis=1)
cd19_abe8e_low_count_df = pd.concat([count_result[0] for count_result in cd19_abe8e_low_count_result_list], axis=1)
cd19_abe8e_high_count_df = pd.concat([count_result[0] for count_result in cd19_abe8e_high_count_result_list], axis=1)

cd19_evocda_presort_count_df = pd.concat([count_result[0] for count_result in cd19_evocda_presort_count_result_list], axis=1)
cd19_evocda_low_count_df = pd.concat([count_result[0] for count_result in cd19_evocda_low_count_result_list], axis=1)
cd19_evocda_high_count_df = pd.concat([count_result[0] for count_result in cd19_evocda_high_count_result_list], axis=1)
cd19_plasmid_series = cd19_abe8e_plasmid_count_result_list[0][0]

count_df = pd.concat([cd19_abe8e_presort_count_df, cd19_abe8e_low_count_df, cd19_abe8e_high_count_df, cd19_evocda_presort_count_df, cd19_evocda_low_count_df, cd19_evocda_high_count_df, cd19_plasmid_series], axis=1)